In [1]:
import torch
import torch.nn as nn
import torch.optim as optim

import numpy as np
from tqdm import tqdm
from utils import *
from network import CharRNN

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [3]:
with open('data/internet_archive_scifi_v3.txt') as f:
    text = f.read()

In [4]:
criterion = nn.CrossEntropyLoss()

# check if there are any checkpoint available from previous runs, else start from zero
try:
    checkpoint = torch.load('models/char_rnn_ckpt.pth', map_location=device)
    model = CharRNN(
        tokens=checkpoint['tokens'],
        n_hidden=checkpoint['n_hidden'],
        n_layers=checkpoint['n_layers'],
        ).to(device)
    model.load_state_dict(checkpoint['model'])
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor=1/3, patience=8, verbose=True)
    optimizer.load_state_dict(checkpoint['optimizer'])
    scheduler.load_state_dict(checkpoint['scheduler'])
    start_epoch = checkpoint['epoch'] + 1
    val_loss_min = checkpoint['val_loss']
    print(f'checkpoint found, training will start from epoch {start_epoch}')
    
    encoding = np.array([model.char2int[ch] for ch in text[580:]])
    
except:
    chars, encoding = tokenize(text[580:])  # scifi text starts from character no '580'
    model = CharRNN(tokens=chars).to(device)
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor=1/3, patience=8, verbose=True)
    start_epoch = 0
    val_loss_min = np.Inf
    print(f'no checkpoint found, training will start from epoch {start_epoch}')

print(model)

# create training and validation data
val_frac = 0.01  # 1% of entire dataset (~1.5M chars)
val_idx = int(len(encoding)*(1-val_frac))
data, val_data = encoding[:val_idx], encoding[val_idx:]

no checkpoint found, training will start from epoch 0
CharRNN(
  (lstm): LSTM(75, 512, num_layers=2, batch_first=True, dropout=0.5)
  (dropout): Dropout(p=0.5, inplace=False)
  (fc): Linear(in_features=512, out_features=75, bias=True)
)


In [5]:
seq_length = 64
batch_size = 512
num_epochs = 32
n_chars = len(model.chars)
num_train_batches = len(data)//(batch_size*seq_length)
num_val_batches = len(val_data)//(batch_size*seq_length)

for epoch in range(start_epoch, num_epochs):
    model.train()
    train_loss = 0

    h = model.init_hidden(batch_size)  # initialize hidden state
    loop = tqdm(get_batches(data, batch_size, seq_length), total=num_train_batches)

    for x, y in loop:
        x = one_hot_encode(x, n_chars)  # one hot encode the data
        inputs, targets = torch.from_numpy(x).to(device), torch.from_numpy(y).to(device)
        # creates new variables for the hidden state, otherwise
        # the optimizer would backprop through the entire training history
        h = tuple([each.data for each in h])
        model.zero_grad()  # zero out accumulated gradients
        outputs, h = model(inputs, h)
        loss = criterion(outputs, targets.view(batch_size*seq_length).long())
        loss.backward()
        # clip gradients, prevents exploding gradient problem in RNNs
        nn.utils.clip_grad_norm_(parameters=model.parameters(), max_norm=5)
        optimizer.step()

        train_loss += loss.item()
        loop.set_description(f'Epoch [{epoch+1:2d}/{num_epochs}]')
        loop.set_postfix(loss=loss.item())


    model.eval()
    val_loss = 0
    with torch.no_grad():
        val_h = model.init_hidden(batch_size)
        for x, y in get_batches(val_data, batch_size, seq_length):
            x = one_hot_encode(x, n_chars)
            inputs, targets = torch.from_numpy(x).to(device), torch.from_numpy(y).to(device)
            outputs, val_h = model(inputs, val_h)
            loss = criterion(outputs, targets.view(batch_size*seq_length).long())
            val_loss += loss.item()


    scheduler.step(val_loss)
    print(f'\n\t\tavg_train_loss={train_loss/num_train_batches:.4f}, avg_val_loss={val_loss/num_val_batches:.4f}')

    # save the model if the validation loss has decreased
    if val_loss <= val_loss_min:
        print(f'\t\tval_loss decreased ({val_loss_min:.4f} --> {val_loss:.4f}) saving model...\n')
        checkpoint = {
            'epoch': epoch,
            'n_hidden': model.n_hidden,
            'n_layers': model.n_layers,
            'tokens': model.chars,
            'model': model.state_dict(),
            'optimizer': optimizer.state_dict(),
            'scheduler': scheduler.state_dict(),
            'val_loss': val_loss
            }
        torch.save(checkpoint, 'models/char_rnn_ckpt.pth')
        val_loss_min = val_loss


Epoch [ 1/32]: 100%|██████████| 4511/4511 [16:54<00:00,  4.45it/s, loss=1.45]



		avg_train_loss=1.6226, avg_val_loss=1.2837
		val_loss decreased (inf --> 57.7671) saving model...



Epoch [ 2/32]: 100%|██████████| 4511/4511 [16:56<00:00,  4.44it/s, loss=1.39]



		avg_train_loss=1.3518, avg_val_loss=1.2196
		val_loss decreased (57.7671 --> 54.8805) saving model...



Epoch [ 3/32]: 100%|██████████| 4511/4511 [16:57<00:00,  4.43it/s, loss=1.37]


...


		avg_train_loss=1.1995, avg_val_loss=1.1034
		val_loss decreased (49.7274 --> 49.6527) saving model...



Epoch [31/32]: 100%|██████████| 4511/4511 [18:14<00:00,  4.12it/s, loss=1.29]



		avg_train_loss=1.1986, avg_val_loss=1.1025
		val_loss decreased (49.6527 --> 49.6115) saving model...



Epoch [32/32]: 100%|██████████| 4511/4511 [18:09<00:00,  4.14it/s, loss=1.29]



		avg_train_loss=1.1978, avg_val_loss=1.1019
		val_loss decreased (49.6115 --> 49.5843) saving model...



In [ ]:
# save only required stuff which will be needed during inference
checkpoint = {
            'n_hidden': model.n_hidden,
            'n_layers': model.n_layers,
            'tokens': model.chars,
            'model': model.state_dict()
            }
torch.save(checkpoint, 'models/char_rnn.pth')